In [1]:
import pandas as pd
import numpy as np
import re
from joblib import dump, load
from sklearn.metrics.pairwise import manhattan_distances, euclidean_distances

In [2]:
data = pd.read_csv("final_database.csv")

In [857]:
'''
data['titre_et_biographie'] = pd.NA

for i in range(0, data.shape[0]):
    if type(data["titre"][i]) == str:
        text_titre = data["titre"][i]
    else:
        text_titre = ""
    
    text_biographie = data["biographie"][i]
    
    data["titre_et_biographie"][i] = str(text_biographie) + ' / ' + str(text_titre)

'''

'\ndata[\'titre_et_biographie\'] = pd.NA\n\nfor i in range(0, data.shape[0]):\n    if type(data["titre"][i]) == str:\n        text_titre = data["titre"][i]\n    else:\n        text_titre = ""\n    \n    text_biographie = data["biographie"][i]\n    \n    data["titre_et_biographie"][i] = str(text_biographie) + \' / \' + str(text_titre)\n\n'

In [862]:
'''
data['text_to_analyze'] = pd.NA

for i in range(0,data.shape[0]):
    
    if type(data["titre"][i]) == str:
        text_titre = data["titre"][i]
    else:
        text_titre = ""
        
    if type(data["description_of_posts"][i]) == str:
        text_posts = data["description_of_posts"][i]
    else:
        text_posts = ""
    
    text_biographie = data["biographie"][i]
    
    data["text_to_analyze"][i] = text_biographie + ' / ' + text_titre + ' / ' + text_posts

'''

'\ndata[\'text_to_analyze\'] = pd.NA\n\nfor i in range(0,data.shape[0]):\n    \n    if type(data["titre"][i]) == str:\n        text_titre = data["titre"][i]\n    else:\n        text_titre = ""\n        \n    if type(data["description_of_posts"][i]) == str:\n        text_posts = data["description_of_posts"][i]\n    else:\n        text_posts = ""\n    \n    text_biographie = data["biographie"][i]\n    \n    data["text_to_analyze"][i] = text_biographie + \' / \' + text_titre + \' / \' + text_posts\n\n'

In [860]:
"""
data["email"] = pd.NA

for i in range(0, data.shape[0]):
    bio = data["biographie"][i]
    emails = re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", bio)
    if len(emails)>0:
        emails = ",".join(emails)
        data["email"][i] = emails
    else:
        pass
        
"""

'\ndata["email"] = pd.NA\n\nfor i in range(0, data.shape[0]):\n    bio = data["biographie"][i]\n    emails = re.findall(r"[a-z0-9\\.\\-+_]+@[a-z0-9\\.\\-+_]+\\.[a-z]+", bio)\n    if len(emails)>0:\n        emails = ",".join(emails)\n        data["email"][i] = emails\n    else:\n        pass\n        \n'

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1878 entries, 0 to 1877
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   username              1878 non-null   object
 1   publications          1878 non-null   object
 2   followers             1878 non-null   int64 
 3   abonnements           1878 non-null   object
 4   titre                 1180 non-null   object
 5   biographie            1878 non-null   object
 6   description_of_posts  1878 non-null   object
 7   email                 180 non-null    object
 8   titre_et_biographie   1878 non-null   object
 9   text_to_analyze       1878 non-null   object
dtypes: int64(1), object(9)
memory usage: 146.8+ KB


# Starting features extracting

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [62]:
stop_words_list = []

with open("stop_words_en.txt") as f:
    for line in f:
        stop_words_list.append(line.removesuffix("\n"))
    f.close()

with open("stop_words_fr.txt") as f:
    for line in f:
        stop_words_list.append(line.removesuffix('\n'))
    f.close()
    
stop_words_list.append("photo")
stop_words_list.append("may")
stop_words_list.append('april')
stop_words_list.append("2022")
stop_words_list.append('2021')
stop_words_list.append('faire')
stop_words_list.append('image')
stop_words_list.append('march')
stop_words_list.append('paris')
stop_words_list.append('france')
stop_words_list.append('people')
stop_words_list.append('person')
stop_words_list.append('personne')
stop_words_list.append('tagging')
stop_words_list.append('vie')
stop_words_list.append('jour')
stop_words_list.append('temps')
stop_words_list.append('lien')
stop_words_list.append('bio')
stop_words_list.append('ans')
stop_words_list.append('post')
stop_words_list.append('vidéo')
stop_words_list.append('veux')
stop_words_list.append('mai')
stop_words_list.append('mois')
stop_words_list.append('avril')
stop_words_list.append('shared')
stop_words_list.append('jamais')
stop_words_list.append('jours')
stop_words_list.append('partage')
stop_words_list.append('instagram')
stop_words_list.append('petit')
stop_words_list.append('indoor')
stop_words_list.append('compte')
stop_words_list.append('commentaire')
stop_words_list.append("semaine")
stop_words_list.append("semaines")
stop_words_list.append('abonne')
stop_words_list.append('monde')
stop_words_list.append('prendre')
stop_words_list.append('faut')
stop_words_list.append('february')
stop_words_list.append('voir')
stop_words_list.append('january')
stop_words_list.append('bonne')
stop_words_list.append('follow')
stop_words_list.append('déjà')
stop_words_list.append("outdoors")
stop_words_list.append("100")
stop_words_list.append("oui")
stop_words_list.append("non")
stop_words_list.append("mieux")
stop_words_list.append('gmail')
stop_words_list.append('contact')
stop_words_list.append('000')
stop_words_list.append('rejoins')
stop_words_list.append('grâce')
stop_words_list.append('devenir')
stop_words_list.append('deviens')
stop_words_list.append('gratuit')
stop_words_list.append('standing')
stop_words_list.append('coach')
stop_words_list.append('coaching')
stop_words_list.append('cartoon')
stop_words_list.append('enregistre')
stop_words_list.append('chocolat')

for i in range(0,31):
    if i<10:
        stop_words_list.append('0'+str(i))
    else:
        stop_words_list.append(str(i))

# Vectorizer titre et biographie

In [63]:
X = data["titre_et_biographie"].values

In [64]:
vectorizer = CountVectorizer(stop_words=stop_words_list, max_features=150)
vectorizer.fit(X)

c:\Users\Abdallah\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'daren', 'hadn', 'herse', 'himse', 'itse', 'mayn', 'mightn', 'mustn', 'myse', 'needn', 'oughtn', 'quelqu', 'shan'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


CountVectorizer(max_features=150,
                stop_words=["'ll", "'tis", "'twas", "'ve", '10', '39', 'a',
                            "a's", 'able', 'ableabout', 'about', 'above',
                            'abroad', 'abst', 'accordance', 'according',
                            'accordingly', 'across', 'act', 'actually', 'ad',
                            'added', 'adj', 'adopted', 'ae', 'af', 'affected',
                            'affecting', 'affects', 'after', ...])

In [65]:
vectorized_input = vectorizer.transform(X)

In [66]:
dump(vectorizer, 'vectorize_bio_et_titre.joblib')

['vectorize_bio_et_titre.joblib']

In [67]:
inv_transform = vectorizer.inverse_transform(vectorized_input)

In [68]:
inv_transform

[array(['créateur', 'digital', 'ligne'], dtype='<U16'),
 array(['produit', 'service'], dtype='<U16'),
 array(['bourse', 'business', 'investissement', 'mindset', 'motivation'],
       dtype='<U16'),
 array(['aide', 'clients', 'entrepreneurs', 'gagner', 'marketing'],
       dtype='<U16'),
 array(['crypto', 'finance', 'formation', 'libre'], dtype='<U16'),
 array(['bourse', 'finance'], dtype='<U16'),
 array(['financière', 'formation', 'infos'], dtype='<U16'),
 array(['blogueur', 'immobilier', 'investissement'], dtype='<U16'),
 array(['créateur', 'digital'], dtype='<U16'),
 array(['gestion'], dtype='<U16'),
 array(['abonnés', 'objectif', 'vidéos'], dtype='<U16'),
 array(['objectif', 'youtube'], dtype='<U16'),
 array([], dtype='<U16'),
 array(['développer', 'formation'], dtype='<U16'),
 array(['founder', 'produit', 'service', 'sport'], dtype='<U16'),
 array([], dtype='<U16'),
 array(['business'], dtype='<U16'),
 array(['entreprise', 'média'], dtype='<U16'),
 array([], dtype='<U16'),
 array([

In [69]:
key_words = []
for arr in inv_transform:
    arr = list(arr)
    arr = sorted(arr)
    arr = ','.join(arr)
    key_words.append(arr)

In [70]:
data['key_words_bio_et_titre'] = key_words

In [71]:
key_words = np.array(key_words)
key_words[key_words==''].shape

(233,)

In [72]:
tfid = TfidfTransformer()

In [73]:
tfid.fit(vectorized_input)

TfidfTransformer()

In [74]:
dump(tfid, 'tfid_bio_et_titre.joblib')

['tfid_bio_et_titre.joblib']

In [959]:
M = tfid.transform(vectorized_input)

In [960]:
test_vector = vectorizer.transform(['Bonjour à toi jeune entrepreneur, si tu veux plus d argent souscrit à mon abonnement gratuit'])

In [961]:
vectorizer.inverse_transform(test_vector)

[array(['argent', 'entrepreneur'], dtype='<U16')]

In [962]:
test_tfid_vector = tfid.transform(test_vector)

In [963]:
from scipy.spatial import distance

null_vector = np.zeros(test_tfid_vector.toarray().shape)
null_distance = distance.cityblock(test_tfid_vector.toarray(), null_vector)

In [964]:
distances = manhattan_distances(M.toarray(), test_tfid_vector.reshape(1,M.toarray().shape[1]))
distances[distances==null_distance]=np.max(distances)

In [965]:
np.sort(distances.reshape(distances.shape[0]))

array([0.        , 0.98596752, 1.35861525, ..., 4.62971278, 4.62971278,
       4.62971278])

# Vectorizer for posts

In [75]:
X_posts = data['description_of_posts'].values

In [76]:
vectorizer = CountVectorizer(stop_words=stop_words_list, max_features=35)
vectorizer.fit(X_posts)

CountVectorizer(max_features=35,
                stop_words=["'ll", "'tis", "'twas", "'ve", '10', '39', 'a',
                            "a's", 'able', 'ableabout', 'about', 'above',
                            'abroad', 'abst', 'accordance', 'according',
                            'accordingly', 'across', 'act', 'actually', 'ad',
                            'added', 'adj', 'adopted', 'ae', 'af', 'affected',
                            'affecting', 'affects', 'after', ...])

In [77]:
from joblib import dump, load

dump(vectorizer, 'vectorize_posts.joblib')

['vectorize_posts.joblib']

In [78]:
vectorized_input = vectorizer.transform(X_posts)

In [79]:
inv_transform = vectorizer.inverse_transform(vectorized_input)

In [80]:
inv_transform

[array(['entrepreneur', 'nature'], dtype='<U22'),
 array(['corps', 'food'], dtype='<U22'),
 array(['argent', 'business', 'confianceensoi', 'investissement',
        'mindset'], dtype='<U22'),
 array(['besoin', 'business', 'conseils', 'corps', 'moment', 'motivation'],
       dtype='<U22'),
 array(['argent', 'crypto'], dtype='<U22'),
 array(['argent', 'business', 'investir', 'investissement', 'motivation',
        'perdre'], dtype='<U22'),
 array(['argent', 'child', 'formation', 'nature', 'savoir'], dtype='<U22'),
 array(['argent', 'conseils', 'investissement', 'savoir'], dtype='<U22'),
 array(['moment', 'savoir', 'travail'], dtype='<U22'),
 array(['entrepreneur'], dtype='<U22'),
 array(['argent', 'confianceensoi', 'contenu', 'developpementpersonnel',
        'motivation'], dtype='<U22'),
 array(['confianceensoi', 'developpementpersonnel', 'entrepreneur', 'food',
        'investissement', 'motivation'], dtype='<U22'),
 array(['fitness'], dtype='<U22'),
 array(['business', 'child', 'inves

In [109]:
key_words = []
for arr in inv_transform:
    arr = list(arr)
    arr = sorted(arr)
    arr = ','.join(arr)
    key_words.append(arr)

In [110]:
key_words = np.array(key_words)
key_words[key_words==""].shape

(297,)

In [86]:
data['key_words_posts'] = key_words

In [87]:
tfid = TfidfTransformer()

In [88]:
tfid.fit(vectorized_input)

TfidfTransformer()

In [89]:
from joblib import dump, load

dump(tfid, 'tfid_posts.joblib')

['tfid_posts.joblib']

In [978]:
M = tfid.transform(vectorized_input)

In [979]:
test_vector = vectorizer.transform(['Bonjour à toi jeune entrepreneur, si tu veux plus d argent souscrit à mon abonnement gratuit'])

In [980]:
vectorizer.inverse_transform(test_vector)

[array(['argent', 'entrepreneur'], dtype='<U22')]

In [981]:
test_tfid_vector = tfid.transform(test_vector)

In [982]:
from scipy.spatial import distance

null_vector = np.zeros(test_tfid_vector.toarray().shape)
null_distance = distance.cityblock(test_tfid_vector.toarray(), null_vector)

In [983]:
distances = manhattan_distances(M.toarray(), test_tfid_vector.reshape(1,M.shape[1]))
distances[distances==null_distance]=1000

In [984]:
np.argsort(distances.reshape(distances.shape[0]))[:10]

array([1127,  237,   30,  115,  386,    9,  809, 1569,  118,  634],
      dtype=int64)